## Template Checker

In [49]:

# import configparser

# def ini_to_config(ini_file):
    
#     def rep_txt(b_in):
#         b_out = b_in.lstrip('\n').rstrip('\n').replace('\t','').replace('%','%%').replace(',\n',',').replace('\n\n','\n').replace(';\n',';')
#         sec = b_out.split('\n',1)[0]
#         b_out = '['+sec.split("'")[1]+']\n'+b_out.replace("'",'').replace('{',"{'").replace('}',"'}")
#         return(b_out)

#     config = configparser.ConfigParser()
#     config.optionxform = str

#     with open(ini_file,'r',encoding='utf8') as f:
#         ini = ''
#         for line in f.readlines():
#             l = line.rstrip()
#             if l.endswith(','):
#                 ini = ini+l
#             else:
#                 ini = ini+l+'\n'
    
#     header = ini.split('[Trace]',1)[0]
#     header = ('\n').join([h for h in header.split('\n') if h.lstrip().startswith('%')==False])
#     items = [rec.replace('%','%%') for rec in header.split('\n') if '=' in rec]
#     default = '[Metadata]\n'
#     for row in items:
#         default += row.replace("'",'') + '\n'
#     config.read_string(default)
#     blocks = [rep_txt(b.split('[End]')[0]) for b in ini.split('[Trace]') if '[End]' in b]
#     for blk in blocks:
#         # try:
#         print(blk)
#         config.read_string(blk)
#         # except:
#         #     print()
#         #     print('Could not convert:')
#         #     print(blk)
#         #     print()

#     # Remove matlab comments but keep % when is a unit
#     for sec in config.sections():
#         for key,value in config[sec].items():
#             if '%' in value and len(value)>1:
#                 config[sec][key] = value.split('%')[0]
#             if '%' in key:
#                 print(f'Confirm removal of {sec}:{key} is correct, it was commented out in input file')
#                 del(config[sec][key])
#     return(config,header)

# def config_to_ini(config,header=''):
#     ini=header
#     for sec in config.sections():
#         if sec != 'Metadata':
#             ini+="[Trace]\n"#\tvariableName = '{sec}'\n"
#             for key,value in config[sec].items():
#                 if len(value)==0:
#                     value="''"
#                 elif value[0] not in ["'",'[','{']:
#                     value=f"'{value}'"
#                 ini+=f"\t{key} = {value}"+'\n' 
#             ini+='[End]\n\n'
#     return(ini)

# # run_BB_db_update>fr_automated_cleaning>fr_cleaning_siteyear>read_data>read_ini_file

In [54]:

import itertools
import os
import configparser


class format_ini():
    def __init__(self):
        self.config = configparser.ConfigParser()
        self.config.optionxform = str

    def ini_to_config(self,ini_file):
        
        with open(ini_file,'r',encoding='utf8') as f:
            ini = ''
            for line in f.readlines():
                l = line.rstrip().lstrip()
                if l.startswith('%')==False:
                    l = l.replace("{'",'{"').replace("'}",'"}').replace(";'",';"')

                    L = l.split("'")
                    # Remove and inline trailing comments
                    L[-1]=L[-1].split('%')[0]
                    l = ("").join(L)
                    l = l.replace('%','%%')
                    l = l.replace('{"',"{'").replace('"}',"'}")
                    l = l.rstrip()
                    if l != '':
                        if l.endswith(',') or l.endswith(';') or l.endswith('...') or l.endswith('end'):
                            if l.endswith(';') or l.endswith('end'):
                                l = l+','
                            ini = ini+l
                        else:
                            ini = ini+l.replace(';"',';')+'\n'

            
        header = ini.split('[Trace]',1)[0]
        header = ('\n').join([h for h in header.split('\n') if h.lstrip().startswith('%')==False])
        items = [rec for rec in header.split('\n') if '=' in rec]
        default = '[Metadata]\n'
        for row in items:
            default += row.replace("'",'') + '\n'
        self.config.read_string(default)
        blocks = [self.get_sec(b.split('[End]')[0]) for b in ini.split('[Trace]') if '[End]' in b]
        for blk in blocks:
            try:
                self.config.read_string(blk)
            except:
                print('Could Not Parse Block:')
                print(blk)
                pass
        
    def get_sec(self,b_in):
        sec = '['+b_in.split('variableName')[1].split('\n')[0].lstrip('= ')+']\n'+b_in
        return(sec)
    
    
SiteID = 'BB'
Stage = 'First'
ini_file = f'../{SiteID}/{SiteID}_{Stage}Stage_oldFormat.ini'

ini=format_ini()
ini.ini_to_config(ini_file)

with open(f'../{SiteID}/{SiteID}_{Stage}Stage.ini', 'w',encoding='utf8') as configfile:
    ini.config.write(configfile,space_around_delimiters=False)


In [51]:
kfc = configparser.ConfigParser()
kfc.read_string('''
[surface_conductance]

variableName = surface_conductance

Evaluate = available_energy = (NETRAD_1_1_1 - G_1);,surface_conductance = calc_surface_conductance_as_flx(TA_1_1_1,RH_1_1_1, aerodynamic_resistance_scalar, available_energy,LE, PA_1_1_1);

title = Surface conductance
units = mol m^{-2} s^{-1}
minMax = [-5,5]''')
kfc['surface_conductance']['units']

'mol m^{-2} s^{-1}'

In [52]:
sentence = 'Mary had a little lamb'
sentence.count('a')

4

In [53]:


V = ['Met','Met/Manual','Flux']
Y = [y for y in range(2023,2025)]


SiteID = 'BB'
Stage = 'SecondStage'
ini_file = f'../{SiteID}/{SiteID}_{Stage}_oldFormat.ini'

config,header = ini_to_config(ini_file)

if Stage == 'FirstStage':
  for trace in config.sections():
      if trace != 'Metadata':
        # config[trace]['inputFileName']=config[trace]['inputFileName']
        n=config[trace]['inputFileName'].split("'")[1]
        chk = [os.path.isfile(f'C:/Database/{y}/{SiteID}/{v}/{n}') for y,v in itertools.product(Y,V)]
        if sum(chk) < 1 and '../' not in n:
            print(f'Remove: {trace}')
            config.remove_section(trace)
  # config_to_ini(config,header)



# ini_file = 'BBS_SecondStage_Template.ini'

if Stage == 'SecondStage':
  config,header = ini_to_config(ini_file)
  for trace in config.sections():
      if trace != 'Metadata':
        print(trace)
        chk = [os.path.isfile(f'C:/Database/{y}/{SiteID}/{v}/Clean/{trace}') for y,v in itertools.product(Y,V)]
        if sum(chk) < 1:
            s = config[trace]['Evaluate'].replace("'",'')
            for v in [trace,';','=',' ']:
                s = s.replace(v,'')
            if len(s)<1:
            # if trace not in config[trace]['Evaluate'].split('=',1)[-1]:
                config.remove_section(trace)
                print(f'remove: {trace}')
            else:
                print('Confirm traces exists to evaluate: ',config[trace]['Evaluate'])

with open(f'../{SiteID}/{SiteID}_{Stage}.ini', 'w',encoding='utf8') as configfile:

  config.write(configfile,space_around_delimiters=False)



NameError: name 'ini_to_config' is not defined

In [ ]:
for key,val in config['TA_1_1_1'].items():
    print(key,val)

KeyError: 'TA_1_1_1'

# Setup

A helper function to define the naming conventions and create ini file commands for BBS

In [ ]:
# import pandas as pd


# Eco = ['PSW','PSTS','PSLS']
# Treatment = ['S','R']
# Depth = ['5cm','10cm','30cm']
# # List = []
# Names = {
#     'Clean_Name':[],
#     'Met_Name':[],
#     'Units':[],
#     'minMax':[],
#     'eval':[]
# }
# h = 1
# for e in Eco:
#     for i,t in enumerate(Treatment):
#         Names['Clean_Name'].append(f'Bog_Height_at_Well_{h}')
#         Names['Met_Name'].append(f'N/A')
#         Names['minMax'].append('[0,500]')
#         Names['Units'].append('cm')        
#         Names['eval'].append(f'BB_WTH_Corrections(SiteID,clean_tv,"Bog_Height_at_Well_{h}");')

#         Names['Clean_Name'].append(f'WTD_{h}')
#         Names['Met_Name'].append(f'N/A')
#         Names['minMax'].append('[-100,100]')
#         Names['Units'].append('cm')
#         Names['eval'].append(f'BB_WTH_Corrections(SiteID,clean_tv,"WTD_{h}");')

#         Names['Clean_Name'].append(f'WTD_{h}_1_1')
#         Names['Met_Name'].append(f'Manual/{e}_{t}_WTD_cm')
#         Names['Units'].append('cm')
#         Names['minMax'].append('[0,200]')
#         Names['eval'].append('N/A')

#         Names['Clean_Name'].append(f'Pipe_Height_{h}_1_1')
#         Names['Met_Name'].append(f'Manual/{e}_{t}_Pipe_Height_cm')
#         Names['Units'].append('cm')
#         Names['minMax'].append('[0,200]')
#         Names['eval'].append('N/A')

#         for v,d in enumerate(Depth):
#             Names['Clean_Name'].append(f'TS_{h}_{v+1}_1')
#             Names['Met_Name'].append(f'{e}_{t}_TSoil_{d}')
#             Names['Units'].append('C')
#             Names['minMax'].append('[-5,35]')
#             Names['eval'].append('N/A')
#         h += 1
        
# met = ['PSW_S_VWC','PSW_R_VWC','PSW_S_WTH','N/A']
# clean = ['VWC_1_1_1','VWC_2_1_1','WTH_1_1_1','WTH_1']
# units = ['%','%','cm','cm']
# minMax = ['[0,95]','[0,95]','[0,200]','[0,1000]']
# eval = ['N/A','N/A','N/A',f'BB_WTH_Corrections(SiteID,clean_tv,"WTH_{1}");']
# for met,clean,u,m,e in zip(met,clean,units,minMax,eval):
#     Names['Met_Name'].append(met)
#     Names['Clean_Name'].append(clean)
#     Names['Units'].append(u)
#     Names['minMax'].append(m)
#     Names['eval'].append(e)

# # [datenum(2014,1,1) datenum(2999,1,1)] 
# # 	loggedCalibration = ''  
# # 	currentCalibration = '' 

# df = pd.DataFrame(data=Names)#,index=['Clean_Name','Type']).T
# # df.index.name = 'Met_Name'
# df['loggedCalibration'] = 'N/A'
# df['currentCalibration'] = 'N/A'
# df['FirstStageOnly'] = False
# # df.loc[df['Clean_Name']=='WTH_1_1_1','loggedCalibration']='[1 0 datenum(2013,1,1) datenum(now)];'
# # df.loc[df['Clean_Name']=='WTH_1_1_1','currentCalibration']='[100 0 datenum(2013,1,1) datenum(now)];'

# for val in ['VWC_1_1_1','VWC_2_1_1','WTH_1_1_1']:
#     df.loc[df['Clean_Name']==val,'loggedCalibration']='[1 0 datenum(2013,1,1) ceil(datenum(now))]'
#     df.loc[df['Clean_Name']==val,'currentCalibration']='[100 0 datenum(2013,1,1) ceil(datenum(now))]'

# for i,row in df.iterrows():
#     if 'Manual' in row['Met_Name']:
#         df.loc[i,'FirstStageOnly'] = True

# # df

## Stage 1

In [ ]:
# Template = '''

# [Trace]
# 	variableName = 'Clean_Name'
# 	title = 'See Soil_ini_Helper.ipynb'
# 	originalVariable = 'See Soil_ini_Helper.ipynb'
# 	inputFileName = {'Met_Name'}
# 	inputFileName_dates =[datenum(2023,1,1) datenum(2100,1,1)]
# 	measurementType = 'Met'
# 	units = 'Units' 
# 	instrument = '-'
# 	instrumentSN = ''
# 	calibrationDates = '' 
# 	loggedCalibration = ''
# 	currentCalibration = ''
# 	comments = ''
# 	minMax = [minMax]
# 	clamped_minMax = [minMax]
# 	zeroPt = [-9999]
# 	plotBottomRight = ''
# 	plotBottomLeft  = ''
# 	dependent       = ''
# [End]

# '''

# Stage_1 = '%%%% Data from CR10_Stations'

# for i,row in df.iterrows():
    
#     Temp = Template
#     if row['loggedCalibration'] != 'N/A':
#         logcal = f"loggedCalibration = {row['loggedCalibration']}"
#         curcal = f"currentCalibration = {row['currentCalibration']}"
#         Temp = Temp.replace("loggedCalibration = ''",logcal)
#         Temp = Temp.replace("currentCalibration = ''",curcal)

#     if row['Met_Name'] != 'N/A':
# 	    Stage_1 += Temp.replace('Clean_Name',row['Clean_Name']).replace('Met_Name',row['Met_Name']).replace('Units',row['Units']).replace('[minMax]',row['minMax'])
    
# # with open('BBS_FirstStage_Template.ini') as file:
# #     file = file.read()
# Stage1_Out = stage_1_ini.replace('%%%% Data from CR10_Stations',Stage_1)
    
# with open('BBS_FirstStage.ini','+w',encoding='utf8') as file:
#     file.write(Stage1_Out)
    
	

# config,header = ini_to_config('BBS_FirstStage.ini')


# with open('BBS_FirstStage_Formatted.ini', 'w',encoding='utf8') as configfile:

#   config.write(configfile,space_around_delimiters=False)

## Stage 2

In [ ]:
# Template = '''

# [Trace]
# 	variableName = 'Clean_Name'
# 	Evaluate = 'Clean_Name = Clean_Name;'
# 	title = '-'
# 	units = 'Units'
# 	minMax = [minMax]
# [End]

# '''

# Stage_2 = '%%%% Data from CR10_Stations'

# for i,row in df.iterrows():
#     if row['FirstStageOnly']:
#         pass
#     else:
#         Temp = Template
#         Stage_2 += Temp.replace('Clean_Name',row['Clean_Name']).replace('Met_Name',row['Met_Name']).replace('Units',row['Units']).replace('[minMax]',row['minMax'])
#         if row['eval'] != 'N/A':
#             eval_in = f"{row['Clean_Name']} = {row['Clean_Name']};"
#             eval_out = f"{row['Clean_Name']} = {row['eval']}"
#             Stage_2 = Stage_2.replace(eval_in,eval_out)
    
# # with open('BBS_SecondStage_Template.ini') as file:
# #     file = file.read()
# Stage2_Out = stage_2_ini.replace('%%%% Data from CR10_Stations',Stage_2)
    
# with open('BBS_SecondStage.ini','+w') as file:
#     file.write(Stage2_Out)